In [ ]:
#This notebook contains code for the Variational Auto-Encoder.
#To run the program run all the cells in the order they are written.

In [ ]:
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import metrics
from tensorflow.keras import backend as K 
from tensorflow.keras.datasets import mnist
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.stats import norm

In [ ]:
# Load and Transform Data
np.random.seed(1000)
train_orig = pd.read_csv('train.csv')
test_orig = pd.read_csv('test.csv')
test_orig['label'] = 11
testCols = test_orig.columns.tolist()
testCols = testCols[-1:] + testCols[:-1]
test_orig = test_orig[testCols]
complete = pd.concat([train_orig, test_orig], ignore_index = True)
valid = complete.sample(n = 5000, random_state = 555)
train = complete.loc[~complete.index.isin(valid.index)]
X_train = train.drop(['label'], axis = 1)
X_valid = valid.drop(['label'], axis = 1)
X_train = X_train.astype('float32') / 255.
X_train = X_train.values.reshape(-1,28,28,1)
X_valid = X_valid.astype('float32') / 255.
X_valid = X_valid.values.reshape(-1,28,28,1)
y_train = train['label']
y_valid = valid['label']

In [ ]:
# Encoder model
img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2  
input_img = tensorflow.keras.Input(shape=img_shape)
x = layers.Conv2D(32, 3, padding='same', activation='relu')(input_img)
x = layers.Conv2D(64, 3, padding='same', activation='relu',strides=(2, 2))(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
shape_before_flattening = K.int_shape(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)
z_mu = layers.Dense(latent_dim)(x)
z_log_sigma = layers.Dense(latent_dim)(x)

In [ ]:
# Sampling function and Decoder Model
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim), mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon
z = layers.Lambda(sampling)([z_mu, z_log_sigma])
decoder_input = layers.Input(K.int_shape(z)[1:])
x = layers.Dense(np.prod(shape_before_flattening[1:]),activation='relu')(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)
x = layers.Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2, 2))(x)
x = layers.Conv2D(1, 3, padding='same', activation='sigmoid')(x)
decoder = Model(decoder_input, x)
z_decoded = decoder(z)

In [ ]:
#Plotting figures
plt.figure(1)
plt.subplot(11)
plt.subplot(12)
plt.imshow(X_train[13][:,:,0])
plt.subplot(14)
plt.imshow(X_train[690][:,:,0])
plt.imshow(X_train[2375][:,:,0])
plt.subplot(15)
plt.imshow(X_train[42013][:,:,0])
plt.show()

In [ ]:
#Applying loss function on a custom layer
class LossLayer(tensorflow.keras.layers.Layer):
    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = tensorflow.keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

In [ ]:
y = LossLayer()([input_img, z_decoded])
vae = Model(input_img, y)
vae.compile(optimizer='adam', loss=None, experimental_run_tf_function=False)
vae.summary()
vae.fit(x=X_train, y=None, shuffle=True, epochs=30, batch_size=batch_size, validation_data=(X_valid, None))
valid_noTest = valid[valid['label'] != 11]
X_valid_n = valid_noTest.drop('label', axis=1)
y_valid_n = valid_noTest['label']
X_valid_n = X_valid_n.astype('float32') / 255.
X_valid_n = X_valid_n.values.reshape(-1,28,28,1)
encoder = Model(input_img, z_mu)
x_valid_n_encoded = encoder.predict(X_valid_n, batch_size=batch_size)
plt.figure(figsize=(10, 10))

In [ ]:
plt.scatter(x_valid_n_encoded[:, 0], x_valid_n_encoded[:, 1], c=y_valid_n, cmap='brg')
plt.colorbar()
plt.show()
custom_cmap = matplotlib.cm.get_cmap('brg')
custom_cmap.set_over('black')
x_valid_encoded = encoder.predict(X_valid, batch_size=batch_size)
plt.figure(figsize=(10, 10))
gray_marker = mpatches.Circle(4,radius=0.1,color='black', label='Including Test Data')
plt.legend(handles=[gray_marker], loc = 'best')
plt.scatter(x_valid_encoded[:, 0], x_valid_encoded[:, 1], c=y_valid, cmap=custom_cmap)
plt.clim(0, 9)
plt.colorbar()
plt.show()